In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

In [2]:
%qtconsole

In [3]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [110]:
aa = file_choose()

In [111]:
print aa
cap = cv2.VideoCapture(aa)
print 'numFrames = ' + str(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))

/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaExamples/KalmiaManualTrig_VidsToProcess/20160615_163530_Tail.mp4
numFrames = 877.0


In [68]:
# change directory
newD = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/AnnotatedVideo'
if not os.path.isdir(newD):
    os.mkdir(newD)
    
os.chdir(newD)

In [109]:
# get digitized points

ptsFile = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/DigitizedPollenFromBee/DLTdv5_data_xypts.csv'
digPts = pd.read_csv(ptsFile)
digPts.ix[95:150].head()

,pt1_cam1_X,pt1_cam1_Y,pt2_cam1_X,pt2_cam1_Y,pt3_cam1_X,pt3_cam1_Y
95,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN
97,800.259398,401.502506,NaN,NaN,NaN,NaN
98,747.647870,427.166667,741.231830,439.998747,NaN,NaN
99,700.169173,464.379699,723.266917,464.379699,NaN,NaN


In [107]:
# watch video with points -- will loop 

cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 


while(ctr <= 150):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()

    if ret: # check ! (some webcam's need a "warmup")
        # our operation on frame come here
        trueCol = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        plt.clf()

        # Display the resulting frame
        io.imshow(trueCol)
        
        # add points
        plt.axis([0, frame.shape[1], frame.shape[0], 0])
        plt.axis('off')
        
        plt.gca().set_position([0,0,1,1])
        plt.gca().set_aspect('equal')
        
        plt.scatter(x = digPts.ix[ctr][[0,2,4]], 
            y = trueCol.shape[0]-digPts.ix[ctr][[1,3,5]], 
           s = 500, 
           alpha = 0.5, edgecolor =  '', 
           color = ['#deebf7', '#9ecae1', '#3182bd'])
        
        # save figure
        plt.savefig(str(ctr).zfill(4) + '.png')
        
        
        print ctr
        ctr +=1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [108]:
# use ffmpeg to convert directory to video
stta = time.time()
!ffmpeg -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y pollenAnnotatedVid.mp4

print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:06.04, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 800x600 [SAR 3937:3937 DAR 4:3], 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264